# Lập trình thống kê cơ bản Python

# Bài 3:  Mode: yếu vị

### BS. Lê Ngọc Khả Nhi

Chào các bạn, đây là bài thứ 3 của Series lập trình thống kê cơ bản sử dụng ngôn ngữ Python. Trong bài này, Nhi sẽ hướng dẫn các bạn cách tìm yếu vị (Mode) của array dữ liệu.

Nhắc lại: yếu vị là giá trị của một hay nhiều phần tử có tần suất hiện diện cao nhất trong danh sách. Như vậy có 2 đặc điểm cần chú ý: thứ nhất, yếu vị độc lập với bản chất của biến/đại lượng mà ta đang khảo sát (giá trị không bắt buộc là con số, có thể là chữ, hoặc trộn lẫn cả chữ và số). Thứ 2, yếu vị không bắt buộc là duy nhất, trong một danh sách có thể tìm thấy nhiều yếu vị.

Như vậy, yếu vị hữu dụng hơn Trung bình và Trung vị, khi chúng ta khảo sát biến định tính, rời rạc hoặc phân bố của mẫu có nhiều mode (multimodal).

In [7]:
import numpy as np
import pandas as pd
import statistics as st
from 

Ta tạo 2 array 10 phần tử, 1 là tập hợp các kí tự A,B,C,D, 1 là chuỗi số nguyên:

In [100]:
np.random.seed(123)

str_array = np.random.choice(list('ABCDE'), size = 10)
num_array = np.random.randint(1,10, size = 10)

## Đếm tần suất thủ công

Sau khi chuyển array thành pandas series, ta có thể dùng method value_counts() đê đếm tần suất mỗi giá trị trong series này và từ đó tìm được mode là giá trị có tần suất cao nhất

In [41]:
pd.Series(str_array).value_counts() # mode là B và C

B    3
C    3
D    2
E    1
A    1
dtype: int64

In [101]:
pd.Series(num_array ).value_counts() # mode là 1

1    4
5    2
2    2
8    1
4    1
dtype: int64

Phương pháp khác để đếm tần suất là hàm Counter của module collections có sẵn trong Python, Kết hợp thêm method most_common() là ta có kết quả như pandas:

In [132]:
from collections import Counter

Counter(num_array).most_common()

[(1, 4), (2, 2), (5, 2), (4, 1), (8, 1)]

In [133]:
Counter(str_array).most_common()

[('C', 3), ('B', 3), ('D', 2), ('E', 1), ('A', 1)]

## Viết hàm:

## Cách 1:

In [137]:
def my_mode(x):
        if len(x) < 1:
            print('Không tìm được mode')
            return None
        else:
            return max(set(x), key=list(x).count)

Hàm này chỉ tìm được 1 mode, nếu có từ 2 mode trở lên nó chỉ xuất ra mode cuối cùng mà nó tìm được, thí dụ C của string array

In [138]:
my_mode(str_array)

'C'

Phiên bản rút gọn của hàm trên:

In [18]:
def my_mode(x):
    return max((list(x).count(i), i) for i in set(x))[1]

In [102]:
my_mode(str_array)

'C'

In [103]:
my_mode(num_array)

1

## Cách 2:

Ta viết 1 hàm khác cho phép tìm được nhiều mode:

In [144]:
def my_mode2(x):
    x = list(x)
    rank_1 = max(list(map(x.count, x)))
    return list(set(filter(lambda i: x.count(i) == rank_1, x)))

In [145]:
my_mode2([1,2,3,1,1,2,5])

[1]

In [146]:
my_mode2(str_array)

['C', 'B']

Khi x là 1 chuỗi giá trị bằng nhau, hàm này xem như có 1 mode

In [148]:
my_mode2([1,1,1,1,1,1])

[1]

Tuy nhiên nó không chấp nhận dữ liệu rỗng:

In [147]:
my_mode2([])

ValueError: max() arg is an empty sequence

## Cách 3

Ta viết 1 hàm khác hoạt động trên mọi dữ liệu, bao gồm dữ liệu rỗng

In [152]:
def my_mode3(x):
    x = list(x)
    counts = {v:x.count(v) for v in set(x)}
    modes = sorted(dict(filter(lambda x: x[1] == max(counts.values()), counts.items())).keys())
    return modes

In [153]:
my_mode3([])

[]

In [157]:
my_mode3([1,1,1,1,1,1])

[1]

In [155]:
my_mode3(list('ABBCDC'))

['B', 'C']

In [156]:
my_mode3([1,1,2,2,3])

[1, 2]

## Cách 4:

Đây là 1 hàm ngắn gọn chủ yếu dùng numpy, nhưng không hoạt động trên dữ liệu rỗng, nó chỉ tìm được 1 mode

In [159]:
def my_mode4(x):
    values, counts = np.unique(x, return_counts=True)
    most_common_index = np.argmax(counts) 
    return values[most_common_index]

In [160]:
my_mode4([])

ValueError: attempt to get argmax of an empty sequence

In [161]:
my_mode4(list('ABCAABCCD'))

'A'

## Cách 5

Đây là hàm cho phép tìm ra 1 mode đầu tiên, nhưng không tìm được nhiều mode

In [166]:
def my_mode5(x):
    counter = Counter(x)
    if len(counter) > 1:
        rank_1, rank_2 = counter.most_common(2)
        if rank_1[1] >= rank_2[1]:
            return rank_1[0]
    return "Không tìm được mode"

In [167]:
my_mode5([])

'Không tìm được mode'

In [169]:
my_mode5(list('ABCDA'))

'A'

In [170]:
my_mode5(list('ABCDAB'))

'A'

## Cách 6

Đây là 1 hàm linh hoạt, nó hoạt động được cho cả dữ liệu rỗng, chuỗi dữ liệu giống nhau, nó cũng tìm được nhiều mode

In [171]:
def my_mode6(x):
    count_dict = {}
    for v in x:
        if v in count_dict:
            count_dict[v]+=1
        else:
            count_dict[v]=1
    
    out = [k for k, i in count_dict.items() if i==max(count_dict.values())]
    print(f'Tìm được {len(out)} mode')
    return out

In [172]:
my_mode6([])

Tìm được 0 mode


[]

In [173]:
my_mode6([1,2,3])

Tìm được 3 mode


[1, 2, 3]

In [174]:
my_mode6([1,1,2,3])

Tìm được 1 mode


[1]

In [175]:
my_mode6([1,1,2,2,3])

Tìm được 2 mode


[1, 2]

In [176]:
my_mode6([1,1,1,1,1,1])

Tìm được 1 mode


[1]

In [177]:
my_mode6([1,2,3,1,2,3,1,2,3])

Tìm được 3 mode


[1, 2, 3]

## Cách 7

Đây là 1 hàm rất thủ công và tường minh, với 1 số quy luật do ta chọn như: nếu dữ liệu rỗng thì thông báo và không tìm mode, nếu dữ liệu chỉ có 1 phần tử, thì xuất ra chính nó, nếu dữ liệu là 1 chuỗi giá trị như nhau thì tìm ra 1 mode, nếu chuỗi có n phần tử với tần suất hiện diện như nhau thì kg tìm mode, ngoài ra thì tìm tất cả mode có thể

In [ ]:
def multi_mode(x):
    
    # Nếu x chỉ có 1 phần tử hoặc rỗng, mode = x
    if len(x) <= 1:
        print(f'X chỉ có {len(x)} phần tử')
        return x
    
    # Nếu x chứa nhiều hơn 1 phần tử
    else:
        count_dict = {}
        
        # Đếm tần suất hiện diện từng giá trị bên trong x, gán vào danh sách đếm
        
        for v in x:
            if v not in count_dict:
                count_dict[v] = 1
            else:
                count_dict[v] += 1
        
        # Nếu danh sách đếm chỉ có 1 phần tử (x là 1 chuỗi lặp lại cùng 1 giá trị)      
        
        if len(count_dict) == 1:
            print(f'Tìm được 1 mode duy nhất = {[v]}')
            return [v]
        
        # Nếu danh sách đếm có nhiều hơn 1 phần tử, xác định giá trị i = tần suất cao nhất
        
        else:
            i = 0
            for v in count_dict:
                if i < count_dict[v]:
                    i = count_dict[v]
            
            # Gán tất cả giá trị v có tần suất = i (cao nhất) vào danh sách modes
            modes = []
            n = 0
            for v in count_dict:
                if count_dict[v] == i:
                    modes.append((v, i, ))
                    n += i
            
            # Nếu tổng tần suất n không bằng kích thước x, xuất kết quả
            
            if n != len(x):
                out = [m[0] for m in modes]
                print(f'Tìm được {len(out)} modes')
                return out
            
            # Nếu không, tức là mỗi phần tử chỉ xuất hiện đúng 1 lần: không tìm được mode
            else:
                print('Không tìm được mode')
                return None

In [178]:
multi_mode([])

X chỉ có 0 phần tử


[]

In [179]:
multi_mode([1])

X chỉ có 1 phần tử


[1]

In [180]:
multi_mode([1, 2, 2.5])

Không tìm được mode


nan

In [181]:
multi_mode(list('ABCAACBB'))

Tìm được 2 modes


['A', 'B']

In [182]:
multi_mode([0,0,0,0])

Tìm được 1 mode duy nhất = [0]


[0]

In [183]:
multi_mode(list('ACTTCTGAGGC'))

Tìm được 3 modes


['C', 'T', 'G']

In [184]:
multi_mode(list('AABBBC'))

Tìm được 1 modes


['B']

In [185]:
multi_mode([1, 2, 0, 0.0 ])

Tìm được 1 modes


[0]

## Cách 8: Dùng module statistics

method mode của module statistics cơ bản của Python chỉ tìm được 1 mode và báo lỗi khi có nhiều mode trong dữ liệu

In [162]:
st.mode(num_array)

1

In [110]:
st.mode(str_array)

StatisticsError: no unique mode; found 2 equally common values

## Cách 9: Dùng module stats của scipy

method mode của scipy.stats cho phép tìm được nhiều mode

In [117]:
from scipy import stats

stats.mode(num_array)

ModeResult(mode=array([1]), count=array([4]))

In [118]:
stats.mode(str_array)

ModeResult(mode=array(['B'], dtype='<U1'), count=array([3]))

## Cách 10: Dùng method mode của pandas

method mode của pandas Series cũng cho phép tìm được nhiều mode

In [163]:
pd.Series(num_array).mode()

0    1
dtype: int32

In [164]:
pd.Series(str_array).mode()

0    B
1    C
dtype: object

Bài thực hành đến đây tạm dừng, chúc các bạn thực hành vui và hẹn gặp lại.